In [9]:
import cv2
import sys
import dlib
import numpy as np
from contextlib import contextmanager
import urllib2
from model import get_model
import config

import tensorflow as tf
#tf.disable_eager_execution()
#from tensorflow.python.framework.ops import disable_eager_execution
#disable_eager_execution()

from os import listdir
from os.path import isfile, join

In [10]:
def get_trained_model():
    weights_file = 'bmi_model_weights.h5'
    model = get_model()
    model.load_weights(weights_file)
    return model

In [11]:
def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1, thickness=2):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness)

In [33]:
def run_demo():
    args = sys.argv[1:]
    multiple_targets = '--multiple' in args
    single_or_multiple = 'multiple faces' if multiple_targets else 'single face'
    model = get_trained_model()
    print('Loading model to detect BMI of %s...' % single_or_multiple)

    NUMBER_OF_FRAMES_IN_AVG = 20
    last_seen_bmis = []
    detector = dlib.get_frontal_face_detector()
    
    #grab an array of the image dir
    mypath = "./normalized_images/" 
    norm_img = [mypath+f for f in listdir(mypath) if isfile(join(mypath, f))]
    print(norm_img)

    for image in norm_img:
        img = cv2.imread(image, flags=cv2.IMREAD_COLOR)
        input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_h, img_w, _ = np.shape(input_img)

        detected = detector(input_img, 1)
        faces = np.empty((len(detected), config.RESNET50_DEFAULT_IMG_WIDTH, config.RESNET50_DEFAULT_IMG_WIDTH, 3))
        if len(detected) > 0:
            print("exist")
            for i, d in enumerate(detected):
                x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
                xw1 = max(int(x1 - config.MARGIN * w), 0)
                yw1 = max(int(y1 - config.MARGIN * h), 0)
                xw2 = min(int(x2 + config.MARGIN * w), img_w - 1)
                yw2 = min(int(y2 + config.MARGIN * h), img_h - 1)
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                faces[i, :, :, :] = cv2.resize(img[yw1:yw2 + 1, xw1:xw2 + 1, :], (config.RESNET50_DEFAULT_IMG_WIDTH, config.RESNET50_DEFAULT_IMG_WIDTH)) / 255.00

            predictions = model.predict(faces)
            print(predictions)

            if multiple_targets:
                for i, d in enumerate(detected):
                    label = str(predictions[i][0])
                    draw_label(img, (d.left(), d.top()), label)
            else:
                last_seen_bmis.append(predictions[0])
                if len(last_seen_bmis) > NUMBER_OF_FRAMES_IN_AVG:
                    last_seen_bmis.pop(0)
                elif len(last_seen_bmis) < NUMBER_OF_FRAMES_IN_AVG:
                    continue
                avg_bmi = sum(last_seen_bmis) / float(NUMBER_OF_FRAMES_IN_AVG)
                label = str(avg_bmi)
                draw_label(img, (d.left(), d.top()), label)
                print(avg_bmi)

        cv2.imshow('result', img)
        key = cv2.waitKey(30)

        if key == 27:  # ESC
            break

In [34]:
run_demo()

Loaded weights from age classifier
Loading model to detect BMI of single face...
['./normalized_images/DIG_LOD_2017_Spring.png', './normalized_images/NV_LOD_2016_Summer.png', './normalized_images/C9T-LOD-2015nacs.png', './normalized_images/GGSA_Benji_2018_Split_2.png', './normalized_images/MBR_LOD_2016_Spring.png', './normalized_images/NV_LOD_2016_Summer_2.png', './normalized_images/GGSA_Benji_2019_Split_1.png']
exist
[[22.681356]]
exist
[[23.24748]]
exist
[[22.302496]]
exist
[[25.530092]]
exist
[[24.22936]]
exist
[[21.601984]]
exist
[[20.983566]]


In [24]:


mypath = "./normalized_images/" 
onlyfiles = [mypath+f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

['./normalized_images/DIG_LOD_2017_Spring.png',
 './normalized_images/NV_LOD_2016_Summer.png',
 './normalized_images/C9T-LOD-2015nacs.png',
 './normalized_images/GGSA_Benji_2018_Split_2.png',
 './normalized_images/MBR_LOD_2016_Spring.png',
 './normalized_images/NV_LOD_2016_Summer_2.png',
 './normalized_images/GGSA_Benji_2019_Split_1.png']